### Script For Data Collection
<p>In This Step you have to mannually collect dataset So that it can be used for Training the Classifier</p>

In [ ]:
import os
import cv2

DATA_DIR = './data'
if not os.path.exists(DATA_DIR):
    os.makedirs(DATA_DIR)

number_of_classes = 24
dataset_size = 150

cap = cv2.VideoCapture(0) #You Might Have to Change the Camera Index According to you Device Index. try 1,2 if Not working
 

for j in range(number_of_classes):
    class_dir = os.path.join(DATA_DIR, str(j))
    if not os.path.exists(class_dir):
        os.makedirs(class_dir)

    print('Collecting data for class {:03d}'.format(j))

    while True:
        ret, frame = cap.read()
        if not ret:
            print("Failed to capture frame. Check camera index or connection.")
            cap.release()
            cv2.destroyAllWindows()
            exit()

        cv2.putText(frame, 'Ready? Press "Q" ! :)', (100, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.3, (0, 255, 0), 3, cv2.LINE_AA)
        cv2.imshow('frame', frame)
        if cv2.waitKey(25) == ord('q'):
            break

    counter = 0
    while counter < dataset_size:
        ret, frame = cap.read()
        if not ret:
            print("Failed to capture frame. Check camera index or connection.")
            break
        cv2.imshow('frame', frame)
        cv2.waitKey(25)
        filename = os.path.join(class_dir, '{:03d}.jpg'.format(counter))
        cv2.imwrite(filename, frame)
        counter += 1

cap.release()
cv2.destroyAllWindows()


## Applying MediaPipe to the Collected Image and Saving the Labels And serialising it using Pickle Library

In [ ]:
import os
import pickle
import mediapipe as mp
import cv2
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

data = []
labels = []
for dir_ in os.listdir(DATA_DIR):
    for img_path in os.listdir(os.path.join(DATA_DIR, dir_)):
        data_aux = []

        x_ = []
        y_ = []

        img = cv2.imread(os.path.join(DATA_DIR, dir_, img_path))
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        results = hands.process(img_rgb)
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                for i in range(len(hand_landmarks.landmark)):
                    x = hand_landmarks.landmark[i].x
                    y = hand_landmarks.landmark[i].y

                    x_.append(x)
                    y_.append(y)

                for i in range(len(hand_landmarks.landmark)):
                    x = hand_landmarks.landmark[i].x
                    y = hand_landmarks.landmark[i].y
                    data_aux.append(x - min(x_))
                    data_aux.append(y - min(y_))

            data.append(data_aux)
            labels.append(dir_)

f = open('data.pickle', 'wb')
pickle.dump({'data': data, 'labels': labels}, f)
f.close()

## Code for training the Model

In [ ]:
import pickle
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np

data_dict = pickle.load(open('./data.pickle', 'rb'))

data = np.asarray(data_dict['data'])
labels = np.asarray(data_dict['labels'])

x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, shuffle=True, stratify=labels)

model = RandomForestClassifier()
model.fit(x_train, y_train)
y_predict = model.predict(x_test)

score = accuracy_score(y_predict, y_test)

print('{}% of samples were classified correctly !'.format(score * 100))

f = open('model.p', 'wb')
pickle.dump({'model': model}, f)
f.close()

## Testing the Model RealTime

In [ ]:
# Load the trained model
model_dict = pickle.load(open('./model.p', 'rb'))
model = model_dict['model']

# Initialize Mediapipe Hands
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

hands = mp_hands.Hands(static_image_mode=True, min_detection_confidence=0.3)

labels_dict = {
    '000': 'A', '001': 'B', '002': 'C', '003': 'D', '004': 'E', '005': 'F', '006': 'G', '007': 'H', '008': 'I', '009': 'K',
    '010': 'L', '011': 'M', '012': 'N', '013': 'O', '014': 'P', '015': 'Q', '016': 'R', '017': 'S', '018': 'T', '019': 'U',
    '020': 'V', '021': 'W', '022': 'X', '023': 'Y'
}

# Reverse the dictionary for lookup
labels_dict_reversed = {v: k for k, v in labels_dict.items()}

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to capture frame from camera. Exiting...")
        break
    
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(frame_rgb)

    data_aux = []
    X = []
    Y = []

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(
                frame,  # image to draw
                hand_landmarks,  # model output
                mp_hands.HAND_CONNECTIONS,  # hand connections
                mp_drawing_styles.get_default_hand_landmarks_style(),
                mp_drawing_styles.get_default_hand_connections_style())

            for i in range(len(hand_landmarks.landmark)):
                x = hand_landmarks.landmark[i].x
                y = hand_landmarks.landmark[i].y
                
                X.append(x)
                Y.append(y)

        if X and Y:
            min_x = min(X)
            min_y = min(Y)

            for i in range(len(X)):
                data_aux.append(X[i] - min_x)
                data_aux.append(Y[i] - min_y)

            if len(data_aux) == model.n_features_in_:
                prediction = model.predict([np.asarray(data_aux)])
                prediction_key = str(int(prediction[0])).zfill(3)
                
                if prediction_key in labels_dict:
                    predicted_character = labels_dict[prediction_key]
                    H, W, _ = frame.shape
                    x1 = int(min(X) * W) - 10
                    y1 = int(min(Y) * H) - 10
                    x2 = int(max(X) * W) + 10
                    y2 = int(max(Y) * H) + 10
                    
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 0), 4)
                    cv2.putText(frame, predicted_character, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 1.3, (0, 0, 0), 3, cv2.LINE_AA)
                else:
                    print(f"Prediction {prediction_key} not found in labels_dict")
            else:
                print(f"Feature size mismatch: Expected {model.n_features_in_}, got {len(data_aux)}")

    cv2.imshow('Hand Tracking', frame)
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
